# Xceptionnet Validation

In [ ]:
from google.colab import drive , files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Function to compute and print metrics
def compute_metrics(y_true, y_pred, y_pred_proba, model_name):
    """
    Compute binary classification metrics and print results.
    Input:
        y_true: Ground truth labels (0 for fake, 1 for real)
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (sigmoid output, shape: (n_samples, 1))
        model_name: Name of the model
    Output: Dictionary of metrics
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=1)
    cm = confusion_matrix(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)  # Use y_pred_proba directly for sigmoid output

    print(f"\nMetrics for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'/content/drive/MyDrive/{model_name}_confusion_matrix.png')
    plt.close()

    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': auc
    }

In [ ]:
# Evaluate XceptionNet
def evaluate_xceptionnet(test_data_path, model_path):
    """
    Evaluate XceptionNet model on test dataset.
    Input:
        test_data_path: Path to test dataset directory
        model_path: Path to XceptionNet model
    """
    # Load model
    xception_model = tf.keras.models.load_model(model_path)

    # Load test data with training preprocessing
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_data = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary',
        shuffle=False  # Keep order for consistent evaluation
    )

    # Verify class indices
    print("Test class indices:", test_data.class_indices)

    # Initialize lists for predictions and true labels
    y_true = []
    y_pred = []
    y_pred_proba = []

    # Process test data
    for images, labels in test_data:
        # Get predictions
        preds = xception_model.predict(images)
        y_pred.extend(np.round(preds).astype(int).flatten())  # Binary predictions
        y_pred_proba.extend(preds)  # Probabilities
        y_true.extend(labels.flatten())

        # Break after processing all images (flow_from_directory loops indefinitely)
        if len(y_true) >= test_data.samples:
            break

        # Convert to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_proba = np.array(y_pred_proba)

    # Compute metrics
    metrics = compute_metrics(y_true, y_pred, y_pred_proba, "XceptionNet")

    # Save metrics to CSV
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv('/content/drive/MyDrive/xceptionnet_metrics.csv', index=False)
    print("\nMetrics Summary:")
    print(metrics_df)

In [ ]:
# Run evaluation
if __name__ == "__main__":
    TEST_DATA_PATH = '/content/drive/MyDrive/Verification_Model'  # Your test data path
    MODEL_PATH = '/content/drive/MyDrive/MODELS/4000-Modelaa/xception_deepfake_final4000.h5'  # Your model path
    evaluate_xceptionnet(TEST_DATA_PATH, MODEL_PATH)

Found 1000 images belonging to 2 classes.
Test class indices: {'fake': 0, 'real': 1}
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━

# FDA Verification

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.fft import fft2
from PIL import Image

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Preprocess image (from training code)
def preprocess_image(image_path, size=(224, 224), is_path=True, augment=False):
    """
    Load and preprocess an image for FDA.
    Args:
        image_path (str): Image file path.
        size (tuple): Target size for resizing (height, width).
        is_path (bool): True for file path input.
        augment (bool): Apply data augmentation if True.
    Returns:
        tf.Tensor: Preprocessed image tensor [1, H, W, C].
    """
    try:
        image = Image.open(image_path).convert('RGB')
        image_np = np.array(image, dtype=np.float32)
        image_tf = tf.image.resize(image_np, size, method='bilinear')

        if augment:
            image_tf = tf.image.random_flip_left_right(image_tf)
            angle = tf.random.uniform((), minval=-10, maxval=10) * np.pi / 180
            image_tf = tf.keras.preprocessing.image.apply_affine_transform(
                image_tf.numpy(), theta=angle * 180 / np.pi, fill_mode='nearest'
            )
            image_tf = tf.convert_to_tensor(image_tf)
            image_tf = tf.image.random_brightness(image_tf, max_delta=0.2)
            image_tf = tf.image.random_contrast(image_tf, lower=0.8, upper=1.2)

        mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
        std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
        image_tf = (image_tf / 255.0 - mean) / std
        image_tf = tf.expand_dims(image_tf, axis=0)
        return image_tf
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {str(e)}")
        return None

In [ ]:
# Frequency domain analysis (from training code)
def frequency_domain_analysis(image_tensor, high_freq_ratio=0.5):
    """
    Perform FDA to extract frequency features.
    Args:
        image_tensor (tf.Tensor): Input image tensor [1, H, W, C].
        high_freq_ratio (float): Ratio of high-frequency components to retain.
    Returns:
        tf.Tensor: Flattened frequency feature vector.
    """
    try:
        image_gray = tf.reduce_mean(image_tensor, axis=3, keepdims=True)
        image_np = image_gray[0, :, :, 0].numpy()
        fft_result = fft2(image_np)
        magnitude_spectrum = np.abs(fft_result)
        magnitude_spectrum = np.fft.fftshift(magnitude_spectrum)

        h, w = magnitude_spectrum.shape
        center_h, center_w = h // 2, w // 2
        high_freq_size = int(min(h, w) * high_freq_ratio / 2)
        high_freq_region = magnitude_spectrum[
            center_h - high_freq_size:center_h + high_freq_size,
            center_w - high_freq_size:center_w + high_freq_size
        ]

        features = high_freq_region.flatten()
        features = (features - features.mean()) / (features.std() + 1e-8)
        feature_tensor = tf.convert_to_tensor(features, dtype=tf.float32)
        return feature_tensor
    except Exception as e:
        print(f"Error in FDA: {str(e)}")
        return None


In [ ]:
# Compute and print metrics
def compute_metrics(y_true, y_pred, y_pred_proba, model_name):
    """
    Compute binary classification metrics and print results.
    Input:
        y_true: Ground truth labels (0 for fake, 1 for real)
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (sigmoid output, shape: (n_samples, 1))
        model_name: Name of the model
    Output: Dictionary of metrics
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=1)
    cm = confusion_matrix(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)

    print(f"\nMetrics for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'/content/drive/MyDrive/{model_name}_confusion_matrix.png')
    plt.close()

    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': auc
    }


In [ ]:
# Evaluate FDA model
def evaluate_frequency_model(test_data_path, model_path):
    """
    Evaluate FDA model on test dataset.
    Input:
        test_data_path: Path to test dataset directory
        model_path: Path to FDA model
    """
    # Load model
    freq_model = tf.keras.models.load_model(model_path)

    # Initialize lists for predictions and true labels
    y_true = []
    y_pred = []
    y_pred_proba = []

    # Process test data
    valid_extensions = ['.jpg', '.jpeg', '.png']
    for label, subfolder in [(1, 'real'), (0, 'fake')]:
        subfolder_path = os.path.join(test_data_path, subfolder)
        if not os.path.exists(subfolder_path):
            print(f"ERROR: Subfolder '{subfolder_path}' not found.")
            return

        image_files = [f for f in os.listdir(subfolder_path) if any(f.lower().endswith(ext) for ext in valid_extensions)]
        for image_name in image_files:
            image_path = os.path.join(subfolder_path, image_name)
            try:
                image_tensor = preprocess_image(image_path, size=(224, 224), is_path=True, augment=False)
                if image_tensor is None:
                    continue
                fda_features = frequency_domain_analysis(image_tensor)
                if fda_features is None:
                    continue

                # Predict
                preds = freq_model.predict(fda_features[np.newaxis, :], verbose=0)
                y_pred.append(int(np.round(preds)[0, 0]))
                y_pred_proba.append(preds[0, 0])
                y_true.append(label)
            except Exception as e:
                print(f"Error processing {image_name}: {str(e)}")

    # Convert to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_proba = np.array(y_pred_proba)

    # Compute metrics
    metrics = compute_metrics(y_true, y_pred, y_pred_proba, "Frequency Domain")

    # Save metrics to CSV
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv('/content/drive/MyDrive/frequency_model_metrics.csv', index=False)
    print("\nMetrics Summary:")
    print(metrics_df)

In [ ]:
# Run evaluation
if __name__ == "__main__":
    TEST_DATA_PATH = '/content/drive/MyDrive/Verification_Model'  # Your test data path
    MODEL_PATH = '/content/drive/MyDrive/MODELS/4000-Modelaa/deepfake_classifier.h5'  # Your FDA model path
    evaluate_frequency_model(TEST_DATA_PATH, MODEL_PATH)


Metrics for Frequency Domain:
Accuracy: 0.7320
Precision: 0.7000
Recall: 0.8120
F1-Score: 0.7519
ROC-AUC: 0.8273
Confusion Matrix:
[[326 174]
 [ 94 406]]

Metrics Summary:
              Model  Accuracy  Precision  Recall  F1-Score  ROC-AUC
0  Frequency Domain     0.732        0.7   0.812  0.751852  0.82732


# Combined

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from PIL import Image
from tensorflow.keras.models import load_model, Model

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Preprocess image for XceptionNet
def preprocess_xception_image(image_path, target_size=(299, 299)):
    """
    Load and preprocess an image for XceptionNet.
    Args:
        image_path (str): Image file path.
        target_size (tuple): Target size (height, width).
    Returns:
        np.ndarray: Preprocessed image array [1, H, W, C].
    """
    try:
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.xception.preprocess_input(img_array)  # Note: XceptionNet training used rescale=1./255
        return img_array
    except Exception as e:
        print(f"Error preprocessing Xception image {image_path}: {str(e)}")
        return None

# Preprocess image for FDA
def preprocess_fda_image(image_path, target_size=(112, 112)):
    """
    Load and preprocess an image for FDA.
    Args:
        image_path (str): Image file path.
        target_size (tuple): Target size (height, width).
    Returns:
        np.ndarray: Preprocessed image array [1, 12544].
    """
    try:
        img = tf.keras.preprocessing.image.load_img(image_path, color_mode='grayscale', target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        if len(img_array.shape) == 2:  # Ensure channel dimension
            img_array = np.expand_dims(img_array, axis=-1)  # Add channel: (112, 112) -> (112, 112, 1)
        img_array = np.expand_dims(img_array, axis=0)  # Shape: (1, 112, 112, 1)
        img_array = img_array / 255.0  # Normalize
        img_array = img_array.reshape(img_array.shape[0], -1)  # Flatten to (1, 12544)
        return img_array
    except Exception as e:
        print(f"Error preprocessing FDA image {image_path}: {str(e)}")
        return None

# Extract features
def extract_single_feature(model, img_array):
    """
    Extract features from a model.
    Args:
        model: Keras model for feature extraction.
        img_array: Preprocessed image array.
    Returns:
        np.ndarray: Flattened feature vector.
    """
    try:
        feature = model.predict(img_array, verbose=0)
        return feature.flatten()
    except Exception as e:
        print(f"Error extracting features: {str(e)}")
        return None

# Compute and print metrics
def compute_metrics(y_true, y_pred, y_pred_proba, model_name):
    """
    Compute binary classification metrics and print results.
    Input:
        y_true: Ground truth labels (0 for fake, 1 for real)
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (sigmoid output, shape: (n_samples,))
        model_name: Name of the model
    Output: Dictionary of metrics
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=1)
    cm = confusion_matrix(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)

    print(f"\nMetrics for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'/content/drive/MyDrive/{model_name}_confusion_matrix.png')
    plt.close()

    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': auc
    }

# Evaluate Combined Model
def evaluate_combined_model(test_data_path, xception_model_path, fda_model_path, combined_model_path):
    """
    Evaluate combined model on test dataset.
    Input:
        test_data_path: Path to test dataset directory
        xception_model_path: Path to XceptionNet model
        fda_model_path: Path to FDA model
        combined_model_path: Path to combined model
    """
    # Load models
    try:
        xception_model = load_model(xception_model_path)
        xception_feature_model = Model(
            inputs=xception_model.input,
            outputs=xception_model.layers[-2].output  # Second-to-last layer (256 units)
        )
    except Exception as e:
        print(f"Error loading Xception model: {str(e)}")
        return

    try:
        fda_model = load_model(fda_model_path)
        fda_feature_model = Model(
            inputs=fda_model.input,
            outputs=fda_model.layers[-2].output  # Second-to-last layer (512 units)
        )
    except Exception as e:
        print(f"Error loading FDA model: {str(e)}")
        return

    try:
        combined_model = load_model(combined_model_path)
    except Exception as e:
        print(f"Error loading combined model: {str(e)}")
        return

    # Initialize lists for predictions and true labels
    y_true = []
    y_pred = []
    y_pred_proba = []

    # Process test data
    valid_extensions = ['.jpg', '.jpeg', '.png']
    for label, subfolder in [(1, 'real'), (0, 'fake')]:
        subfolder_path = os.path.join(test_data_path, subfolder)
        if not os.path.exists(subfolder_path):
            print(f"ERROR: Subfolder '{subfolder_path}' not found.")
            return

        image_files = [f for f in os.listdir(subfolder_path) if any(f.lower().endswith(ext) for ext in valid_extensions)]
        for image_name in image_files:
            image_path = os.path.join(subfolder_path, image_name)
            try:
                # Xception features
                xception_img = preprocess_xception_image(image_path, target_size=(299, 299))
                if xception_img is None:
                    continue
                xception_features = extract_single_feature(xception_feature_model, xception_img)
                if xception_features is None:
                    continue
                print(f"Xception features shape for {image_name}: {xception_features.shape}")  # Debug

                # FDA features
                fda_img = preprocess_fda_image(image_path, target_size=(112, 112))
                if fda_img is None:
                    continue
                fda_features = extract_single_feature(fda_feature_model, fda_img)
                if fda_features is None:
                    continue
                print(f"FDA features shape for {image_name}: {fda_features.shape}")  # Debug

                # Predict
                preds = combined_model.predict(
                    [np.expand_dims(xception_features, axis=0), np.expand_dims(fda_features, axis=0)],
                    verbose=0
                )
                y_pred.append(int(np.round(preds)[0, 0]))
                y_pred_proba.append(preds[0, 0])
                y_true.append(label)
            except Exception as e:
                print(f"Error processing {image_name}: {str(e)}")

    # Convert to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_proba = np.array(y_pred_proba)

    # Compute metrics
    metrics = compute_metrics(y_true, y_pred, y_pred_proba, "Combined Model")

    # Save metrics to CSV
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv('/content/drive/MyDrive/combined_model_metrics_test.csv', index=False)  # Distinct filename
    print("\nMetrics Summary:")
    print(metrics_df)


In [ ]:
# Run evaluation
if __name__ == "__main__":
    TEST_DATA_PATH = '/content/drive/MyDrive/Verification_Model'  # Your test dataset path
    XCEPTION_MODEL_PATH = '/content/drive/MyDrive/MODELS/4000-Modelaa/xception_deepfake_final4000.h5'
    FDA_MODEL_PATH = '/content/drive/MyDrive/MODELS/4000-Modelaa/deepfake_classifier.h5'  # Updated path
    COMBINED_MODEL_PATH = '/content/drive/MyDrive/MODELS/4000-Modelaa/FF_combined.h5'
    evaluate_combined_model(TEST_DATA_PATH, XCEPTION_MODEL_PATH, FDA_MODEL_PATH, COMBINED_MODEL_PATH)

Xception features shape for real04278.jpg: (256,)
FDA features shape for real04278.jpg: (512,)
Xception features shape for real05561.jpg: (256,)
FDA features shape for real05561.jpg: (512,)
Xception features shape for real06874.jpg: (256,)
FDA features shape for real06874.jpg: (512,)
Xception features shape for real07106.jpg: (256,)
FDA features shape for real07106.jpg: (512,)
Xception features shape for real05006.jpg: (256,)
FDA features shape for real05006.jpg: (512,)
Xception features shape for real05744.jpg: (256,)
FDA features shape for real05744.jpg: (512,)
Xception features shape for real03030.jpg: (256,)
FDA features shape for real03030.jpg: (512,)
Xception features shape for real03652.jpg: (256,)
FDA features shape for real03652.jpg: (512,)
Xception features shape for real05910.jpg: (256,)
FDA features shape for real05910.jpg: (512,)
Xception features shape for real06950.jpg: (256,)
FDA features shape for real06950.jpg: (512,)
Xception features shape for real03132.jpg: (256,)


In [ ]:
import pandas as pd

# Load CSV files
try:
    xception_metrics = pd.read_csv('/content/drive/MyDrive/xceptionnet_metrics.csv')
    print("Loaded XceptionNet metrics")
except FileNotFoundError:
    print("Error: xceptionnet_metrics.csv not found")
    xception_metrics = None

try:
    freq_metrics = pd.read_csv('/content/drive/MyDrive/frequency_model_metrics.csv')
    print("Loaded Frequency Domain metrics")
except FileNotFoundError:
    print("Error: frequency_model_metrics.csv not found")
    freq_metrics = None

try:
    combined_metrics = pd.read_csv('/content/drive/MyDrive/combined_model_metrics_test.csv')
    print("Loaded Combined Model (Test) metrics")
except FileNotFoundError:
    print("Error: combined_model_metrics_test.csv not found")
    combined_metrics = None

# Combine available DataFrames
metrics_list = [df for df in [xception_metrics, freq_metrics, combined_metrics] if df is not None]
if metrics_list:
    all_metrics = pd.concat(metrics_list, ignore_index=True)
    # Save combined metrics
    all_metrics.to_csv('/content/drive/MyDrive/all_model_metrics.csv', index=False)
    print("\nCombined Metrics Summary:")
    print(all_metrics)
else:
    print("Error: No metrics files found. Please run the evaluation scripts first.")

Loaded XceptionNet metrics
Loaded Frequency Domain metrics
Loaded Combined Model (Test) metrics

Combined Metrics Summary:
              Model  Accuracy  Precision  Recall  F1-Score   ROC-AUC
0       XceptionNet     0.853   0.846758   0.862  0.854311  0.937492
1  Frequency Domain     0.732   0.700000   0.812  0.751852  0.827320
2    Combined Model     0.882   0.864504   0.906  0.884766  0.951544


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load combined metrics
all_metrics = pd.read_csv('/content/drive/MyDrive/all_model_metrics.csv')

# Plot bar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.2
x = np.arange(len(metrics))
models = all_metrics['Model'].values

for i, model in enumerate(models):
    values = all_metrics.iloc[i][metrics].values
    ax.bar(x + i*bar_width, values, bar_width, label=model)

ax.set_xlabel('Metrics')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + bar_width*(len(models)-1)/2)
ax.set_xticklabels(metrics)
ax.legend()
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/model_metrics_comparison.png')
plt.close()